In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import xgboost as xgb

In [20]:
df = pd.read_parquet('c:/fomo_data2/complete_for_models.parquet')
df.head()

,locationid,ridership,year,month,day,hour,temperature_2m (°C),windspeed_10m (km/h),winddirection_10m (°),sentiment,events
0,2.0,3.0,2023.0,1,1,15.0,1.2,7.1,66.0,0.0,0
1,2.0,1.0,2023.0,1,1,19.0,2.8,6.9,99.0,0.0,0
2,3.0,9.0,2023.0,1,1,0.0,4.3,9.9,327.0,1.0,0
3,3.0,16.0,2023.0,1,1,1.0,3.9,9.0,331.0,1.0,0
4,3.0,12.0,2023.0,1,1,2.0,3.3,8.9,328.0,1.0,0


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3222684 entries, 0 to 3222683
Data columns (total 11 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   locationid             float64
 1   ridership              float64
 2   year                   float64
 3   month                  int32  
 4   day                    int32  
 5   hour                   float64
 6   temperature_2m (°C)    float32
 7   windspeed_10m (km/h)   float32
 8   winddirection_10m (°)  float32
 9   sentiment              float64
 10  events                 int64  
dtypes: float32(3), float64(5), int32(2), int64(1)
memory usage: 209.0 MB


In [22]:
print(len(df))
index= df.dropna()
print(len(df))

3222684
3222684


In [23]:
# Find the range of values in the 'ridership' column
min_value = df['ridership'].min()
max_value = df['ridership'].max()
print(f"Range of ridership values: {min_value} to {max_value}")

# Describe the ridership column to get basic statistics
description = df['ridership'].describe()
print(description)

Range of ridership values: 0.0 to 65536.0
count    3.222682e+06
mean     1.050523e+03
std      2.616317e+03
min      0.000000e+00
25%      3.500000e+01
50%      2.400000e+02
75%      9.250000e+02
max      6.553600e+04
Name: ridership, dtype: float64


In [12]:

# Splitting data into features and target
X = df.drop('ridership', axis=1)
y = df['ridership']

# Splitting data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [28]:
# Find the indices of NaN values in y_train
nan_indices = y_train[y_train.isna()].index

# Display the indices and corresponding NaN values
print("Indices of NaN values in y_train:")
print(nan_indices)

# Optionally, you can also display the rows in the dataset corresponding to these NaN indices if needed
print("Rows in y_train with NaN values:")
print(y_train.loc[nan_indices])

Indices of NaN values in y_train:
Index([404895, 404896], dtype='int64')
Rows in y_train with NaN values:
404895   NaN
404896   NaN
Name: ridership, dtype: float64


In [30]:
# Fill NaN values in y_train with the median value
y_train = y_train.fillna(y_train.median())

# Verify that there are no NaNs left
print("Checking for NaNs in cleaned y_train:")
print(y_train.isna().sum())

Checking for NaNs in cleaned y_train:
0


In [31]:
xgb_reg = xgb.XGBRegressor(objective='reg:squarederror', seed=42)

# Train the model
xgb_reg.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [32]:
y_pred = xgb_reg.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [33]:
evaluation_metrics = pd.DataFrame({
    'Metric': ['Mean Absolute Error', 'Mean Squared Error', 'R-squared'],
    'Value': [mae, mse, r2]
})
print(evaluation_metrics)

                Metric         Value
0  Mean Absolute Error  4.302352e+02
1   Mean Squared Error  1.395795e+06
2            R-squared  7.962731e-01


In [34]:
print(len(df))

3222684


In [35]:
df.nunique()

locationid                 260
ridership                27710
year                         2
month                       12
day                         31
hour                        24
temperature_2m (°C)        444
windspeed_10m (km/h)       340
winddirection_10m (°)      360
sentiment                   69
events                       2
dtype: int64

In [36]:
import pickle
with open('c:/fomo_data2/xgb_model.pkl', 'wb') as file:
    pickle.dump(xgb_reg, file)